# Setup

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import os
import pandas as pd

from plotting import *
from prices import *
from imputation import *
from inflation import *
from cost import *

In [5]:
results_dir = 'results/sensitivity_analysis/'
os.makedirs(results_dir, exist_ok=True)

# Cost estimation

In [6]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation()

/Users/bencottier/Projects/training_cost_2/training-cost-trends/data.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frontier_pcd_df.loc[:, 'Training compute (FLOP)'] = pd.to_numeric(


In [7]:
cost_df = estimate_costs(frontier_pcd_df, hardware_df, price_df)
cost_df

==== System: Yuan 1.0 ====
Training time is required but no value found

==== System: OpenAI TI7 DOTA 1v1 ====
Training time is required but no value found

==== System: Claude 2 ====
Training time is required but no value found

==== System: Turing-NLG ====
Training time is required but no value found

==== System: GPT-3.5 (text-davinci-003) ====
Training time is required but no value found

==== System: AlphaGo Master ====
Training time is required but no value found

==== System: Minerva (540B) ====
Trying Google TPU v4 at 2022-04-01 00:00:00
Trying Google Cloud, Price per chip-hour (3-year CUD)
Found price: 1.45

==== System: FTW ====
Training time is required but no value found

==== System: AlphaGo Zero ====
Trying Google TPU v1 at 2017-07-30 00:00:00
Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (1-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (on-demand)
Could not find price
==== Sys

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Open-source,Link,Citations,Reference,...,Training cost trends,Training cloud compute vendor,Training data center,Archived links,Batch size,Batch size notes,Datasets,Organization categorization,Training time (chip hours),Cost
358,Yuan 1.0,Language,Language modelling,"Shaohua Wu, Xudong Zhao, Tong Yu, Rongguo Zhan...",SOTA Improvement,"""The zero-shot average scores of both LM and P...",NaN,https://arxiv.org/abs/2110.04725,37.0,Yuan 1.0: Large-Scale Pre-trained Language Mod...,...,Yuan 1.0,NaN,NaN,NaN,6881280.0,"Table 2. Batch size 3360, sequence length 2048...",NaN,Industry,NaN,NaN
368,OpenAI TI7 DOTA 1v1,Games,DOTA,NaN,"SOTA Improvement,Historical significance",NaN,NaN,https://openai.com/research/dota-2,0.0,Dota 2,...,OpenAI TI7 DOTA 1v1,NaN,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN
378,Claude 2,Language,Language modelling,NaN,Historical significance,NaN,API accessible,"https://www.anthropic.com/index/claude-2, http...",0.0,NaN,...,Claude 2,NaN,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN
428,Turing-NLG,Language,Text autocompletion,Corby Rosset,SOTA Improvement,"from paper: ""Turing Natural Language Generatio...",NaN,https://www.microsoft.com/en-us/research/blog/...,114.0,Turing-NLG: A 17-billion-parameter language mo...,...,Turing-NLG,NaN,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN
443,GPT-3.5 (text-davinci-003),Language,Language modelling,NaN,"SOTA Improvement,Historical significance,Signi...",NaN,API accessible,https://platform.openai.com/docs/models/gpt-3-5,NaN,NaN,...,GPT-3.5 (text-davinci-003),NaN,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1273,HyperClova,Language,NaN,"Boseop Kim, HyoungSeok Kim, Sang-Woo Lee, Gich...",SOTA Improvement,"""HyperCLOVA with our training configuration sh...",NaN,https://arxiv.org/abs/2109.04650,79.0,What Changes Can Large-scale Language Models B...,...,HyperClova,NaN,NaN,NaN,NaN,"""All models use the mini-batch size of 1,024""....",NaN,"Industry,Industry",658636.8,948436.992
1274,PanGu-Σ,Language,"Code generation,Language modelling","Xiaozhe Ren, Pingyi Zhou, Xinfan Meng, Xinjing...",SOTA Improvement,"""Our experimental findings show that PanGu-{\S...",NaN,https://arxiv.org/abs/2303.10845,28.0,PanGu-Σ: Towards Trillion Parameter Language M...,...,PanGu-Σ,NaN,NaN,NaN,524288.0,"""We train PanGu-Σ with global batch size of 51...",NaN,Industry,1228800.0,NaN
1276,GLM-130B,Language,NaN,NaN,SOTA Improvement,"""GLM-130B achieves an accuracy of 80.2% on zer...",NaN,https://keg.cs.tsinghua.edu.cn/glm-130b/posts/...,60.0,GLM-130B: An open bilingual pre-trained model,...,GLM-130B,NaN,NaN,NaN,NaN,NaN,NaN,Academia,1105920.0,1603584.000
1277,Flamingo,Multimodal,"Visual question answering,Image captioning","Jean-Baptiste Alayrac, Jeff Donahue, Pauline L...",SOTA Improvement,"""For tasks lying anywhere on this spectrum, a ...",NaN,https://arxiv.org/abs/2204.14198,1320.0,Flamingo: a Visual Language Model for Few-Shot...,...,Flamingo,NaN,NaN,NaN,NaN,NaN,"""MultiModal MassiveWeb, LTIP, VTP, ALIGN""",Industry,552960.0,801792.000


In [8]:
print(cost_df['Cost'].notna().sum(), 'of', len(cost_df), 'systems have a cost estimate')

32 of 68 systems have a cost estimate


In [9]:
print(cost_df['Training time (chip hours)'].notna().sum(), 'of', len(cost_df), 'systems have chip hours')

36 of 68 systems have chip hours


# Sensitivity of cloud prices to fallbacks

In [10]:
# TODO: use the vendor mapping from imputation to reduce repetition
org_to_cloud_vendor = {
    'google': 'Google Cloud',
    'deepmind': 'Google Cloud',
    'microsoft': 'Microsoft Azure',
    'openai': 'Microsoft Azure',
}

pcd_hardware_model_colname = 'Training hardware'
price_colname = 'Price per chip-hour (3-year CUD)'

In [15]:
import contextlib

# Load data
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation()

# Use the below to track which fallbacks are still active
fallback_names = [
    'price_date_after',
    'default_vendor',
    'backup_vendor',
    'backup_price_type',
]
# Turn off all fallbacks
fallback_is_active = {fallback: False for fallback in fallback_names}
current_fallback_idx = 0
fallback_name = 'price_date_before'
results = pd.DataFrame(columns=['iteration', 'fallback', 'system', 'price'])
iteration = 0
# Loop
while True:
    # Do price matching
    system_to_price = {}

    for i, row in frontier_pcd_df.iterrows():
        # with contextlib.redirect_stdout(None):
        price_value, price_id = find_price(
            row,
            price_df,
            hardware_df,
            pcd_hardware_model_colname,
            price_colname,
            org_to_cloud_vendor,
            **fallback_is_active,
        )
        if price_value is None:
            continue
        else:
            system_to_price[row['System']] = price_value, price_id
            # Store the prices that matched along with their systems
            results.loc[len(results)] = [iteration, fallback_name, row['System'], price_value]
    if all(fallback_is_active.values()):
        break
    # Remove the rows in price_df which have a 'Price source' in system_to_price
    price_df = price_df[~price_df['Price source'].isin([price_id for _, price_id in system_to_price.values()])]
    # If no prices matched, turn on the next fallback
    if len(system_to_price) == 0:
        print(fallback_names[current_fallback_idx], 'is active')
        fallback_name = fallback_names[current_fallback_idx]
        fallback_is_active[fallback_name] = True
        current_fallback_idx += 1
    else:
        iteration += 1

/Users/bencottier/Projects/training_cost_2/training-cost-trends/data.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frontier_pcd_df.loc[:, 'Training compute (FLOP)'] = pd.to_numeric(


==== System: Yuan 1.0 ====
Training time is required but no value found

==== System: OpenAI TI7 DOTA 1v1 ====
Training time is required but no value found

==== System: Claude 2 ====
Training time is required but no value found

==== System: Turing-NLG ====
Training time is required but no value found

==== System: GPT-3.5 (text-davinci-003) ====
Training time is required but no value found

==== System: AlphaGo Master ====
Training time is required but no value found

==== System: Minerva (540B) ====
Trying Google TPU v4 at 2022-04-01 00:00:00
Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price
==== System: FTW ====
Training time is required but no value found

==== System: AlphaGo Zero ====
Trying Google TPU v1 at 2017-07-30 00:00:00
Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price
==== System: NASv3 (CIFAR-10) ====
Training time is required but no value found

==== System: AlphaZero ====
Training time is required but no value found



In [16]:
results

,iteration,fallback,system,price
0,0,price_date_before,ALIGN,0.90
1,0,price_date_before,AlphaStar,0.90
2,0,price_date_before,GShard (dense),0.90
3,0,price_date_before,Gopher (280B),0.90
4,0,price_date_before,Meena,0.90
5,0,price_date_before,LaMDA,0.90
6,0,price_date_before,T5-11B,0.90
7,0,price_date_before,Switch,0.90
8,0,price_date_before,Gemini Ultra,1.45
9,0,price_date_before,Meta Pseudo Labels,0.90


In [24]:
# Find the index at which each fallback was activated
fallback_idxs = results.groupby('fallback').apply(lambda x: x.index[0])
fallback_idxs

fallback
backup_price_type    45
backup_vendor        31
default_vendor       15
price_date_after     12
price_date_before     0
dtype: int64

In [26]:
import plotly.graph_objects as go

fig = go.Figure()

# We want to plot a line of the price over "time" (i.e. the order in which the systems were processed), for each system
for system in results['system'].unique():
    system_results = results[results['system'] == system]
    # Sort by index
    system_results = system_results.sort_index()
    fig.add_trace(go.Scatter(x=system_results.index, y=system_results['price'], mode='lines+markers', name=system))

# Now we want to plot a vertical line at each fallback
for fallback, idx in fallback_idxs.items():
    # Annotate with the fallback name, rotated -90 degrees and aligned to the top of the y axis
    fig.add_annotation(
        x=idx,
        y=4,
        text=fallback,
        showarrow=False,
        textangle=-90,
        xshift=-20,
        xanchor='left',
        yanchor='top',
    )
    fig.add_shape(
        # Line Vertical
        dict(
            type="line",
            x0=idx,
            y0=0,
            x1=idx,
            y1=4,
            # gray dashed line
            line=dict(
                color="Grey",
                width=2,
                dash="dashdot",
            )
        )
    )

# Layout
fig.update_layout(
    title='Effect of fallbacks on cloud price matching',
    xaxis_title='idx',
    yaxis_title='Price matched to system (USD per chip-hour)',
    # legend_title='System',
    showlegend=False,
)

save_plot(fig, results_dir, 'price_fallback_effect')
fig.show()

# Sensitivity of cost estimates to imputation

In [46]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation()

In [47]:
frontier_pcd_df = frontier_pcd_df.set_index('System')
frontier_pcd_df['System'] = frontier_pcd_df.index

In [50]:
target_col = 'Training hardware'
reference_col = 'Cost'
num_drop = 5
imputer_fn = knn_impute_categorical_column

In [51]:
known_costs_mask = cost_df['Cost'].notna()
# Filter for rows where all of the required information was known
all_known_df = frontier_pcd_df[known_costs_mask]
all_known_df[target_col]

System
GNMT                                     NVIDIA Tesla K80
JFT                                      NVIDIA Tesla K80
AmoebaNet-A (F=448)                     NVIDIA Tesla K40s
IMPALA                                        NVIDIA P100
BigGAN-deep 512x512                         Google TPU v3
RoBERTa Large                NVIDIA Tesla V100 DGXS 32 GB
Megatron-BERT               NVIDIA Tesla V100S PCIe 32 GB
Megatron-LM (8.3B)           NVIDIA Tesla V100 DGXS 32 GB
T5-11B                                      Google TPU v3
AlphaStar                                   Google TPU v3
Meena                                       Google TPU v3
GPT-3 175B (davinci)         NVIDIA Tesla V100 DGXS 32 GB
GShard (dense)                              Google TPU v3
ALIGN                                       Google TPU v3
Megatron-Turing NLG 530B           NVIDIA A100 SXM4 80 GB
Gopher (280B)                               Google TPU v3
GLaM                                        Google TPU v4
LaMDA  

In [53]:
# Drop random values where all of the required information was known
dropped_df, holdout_df = drop_random_values(frontier_pcd_df, target_col, cost_df, reference_col, num_drop=num_drop)
dropped_df[known_costs_mask][target_col]

System
GNMT                                     NVIDIA Tesla K80
JFT                                      NVIDIA Tesla K80
AmoebaNet-A (F=448)                     NVIDIA Tesla K40s
IMPALA                                        NVIDIA P100
BigGAN-deep 512x512                         Google TPU v3
RoBERTa Large                NVIDIA Tesla V100 DGXS 32 GB
Megatron-BERT               NVIDIA Tesla V100S PCIe 32 GB
Megatron-LM (8.3B)           NVIDIA Tesla V100 DGXS 32 GB
T5-11B                                      Google TPU v3
AlphaStar                                   Google TPU v3
Meena                                                 NaN
GPT-3 175B (davinci)         NVIDIA Tesla V100 DGXS 32 GB
GShard (dense)                              Google TPU v3
ALIGN                                       Google TPU v3
Megatron-Turing NLG 530B           NVIDIA A100 SXM4 80 GB
Gopher (280B)                               Google TPU v3
GLaM                                                  NaN
LaMDA  

In [14]:
cost_df_with_imputation = estimate_costs(dropped_df, hardware_df, price_df, impute_pcd_data=True)

==== System: GNMT ====
Trying NVIDIA Tesla K80 at 2016-01-30 00:00:00
Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (1-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (on-demand)
Found price: 1.4

==== System: JFT ====
Trying NVIDIA Tesla K80 at 2017-03-13 00:00:00
Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (1-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (on-demand)
Found price: 1.4

==== System: AmoebaNet-A (F=448) ====
Trying NVIDIA Tesla K80 at 2017-11-29 00:00:00
Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (1-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (on-demand)
Found price: 1.4

==== System: IMPALA ====
Trying NVIDIA P100 at 2017-12-01 20:00:00
Trying Google Cloud, Price per chip-hour (3-ye

In [15]:
cost_df_with_imputation['Training hardware']

System
GNMT                                     NVIDIA Tesla K80
JFT                                      NVIDIA Tesla K80
AmoebaNet-A (F=448)                      NVIDIA Tesla K80
IMPALA                                        NVIDIA P100
BigGAN-deep 512x512                         Google TPU v3
RoBERTa Large                            NVIDIA Tesla K80
Megatron-BERT               NVIDIA Tesla V100S PCIe 32 GB
Megatron-LM (8.3B)           NVIDIA Tesla V100 DGXS 32 GB
T5-11B                                      Google TPU v3
AlphaStar                                   Google TPU v3
Meena                                       Google TPU v3
GPT-3 175B (davinci)         NVIDIA Tesla V100 DGXS 32 GB
GShard (dense)                              Google TPU v3
ALIGN                                       Google TPU v3
Megatron-Turing NLG 530B           NVIDIA A100 SXM4 80 GB
Gopher (280B)                               Google TPU v3
GLaM                                        Google TPU v4
LaMDA  

In [16]:
cost_df_with_imputation['Cost']

System
GNMT                        5.806080e+05
JFT                         1.008000e+05
AmoebaNet-A (F=448)         1.058400e+05
IMPALA                      1.460000e+02
BigGAN-deep 512x512         2.457600e+04
RoBERTa Large               5.283840e+04
Megatron-BERT               7.492494e+04
Megatron-LM (8.3B)          2.025830e+05
T5-11B                      2.220595e+05
AlphaStar                   3.649536e+05
Meena                       6.635520e+05
GPT-3 175B (davinci)        4.297920e+06
GShard (dense)              9.289728e+05
ALIGN                       1.600358e+05
Megatron-Turing NLG 530B    6.209280e+06
Gopher (280B)               3.391488e+06
GLaM                        2.028237e+06
LaMDA                       1.276416e+06
PaLM (540B)                 1.218724e+07
OPT-175B                    7.312896e+05
Minerva (540B)              1.322066e+07
LLaMA-65B                   1.392640e+06
GPT-4                       8.265000e+07
Falcon 180B                 2.565734e+07
Name: Cos

In [17]:
cost_df['Cost']

System
AlphaGo Fan                            NaN
AlphaGo Lee                            NaN
GNMT                          5.806080e+05
NASv3 (CIFAR-10)                       NaN
AlphaGo Master                         NaN
JFT                           1.008000e+05
OpenAI TI7 DOTA 1v1                    NaN
AlphaGo Zero                           NaN
AlphaZero                              NaN
AmoebaNet-A (F=448)           3.985070e+03
IMPALA                        1.460000e+02
ResNeXt-101 32x48d                     NaN
FTW                                    NaN
BigGAN-deep 512x512           2.457600e+04
RoBERTa Large                 1.486848e+05
Megatron-BERT                 7.492494e+04
Megatron-LM (8.3B)            2.025830e+05
T5-3B                                  NaN
T5-11B                        2.220595e+05
AlphaStar                     3.649536e+05
OpenAI Five                            NaN
OpenAI Five Rerun                      NaN
Meena                         6.635520e+05
Turi

In [18]:
# Measure the difference in cost for rows where data was dropped and imputed
holdout_costs_with_imputation = cost_df_with_imputation.loc[holdout_df.index, 'Cost']
holdout_costs = cost_df.loc[holdout_df.index, 'Cost']

holdout_costs_with_imputation

System
AmoebaNet-A (F=448)      105840.0
OPT-175B                 731289.6
RoBERTa Large             52838.4
GShard (dense)           928972.8
GPT-4                  82650000.0
Name: Cost, dtype: float64

In [19]:
holdout_costs

System
AmoebaNet-A (F=448)    3.985070e+03
OPT-175B               1.470705e+06
RoBERTa Large          1.486848e+05
GShard (dense)         9.289728e+05
GPT-4                  8.265000e+07
Name: Cost, dtype: float64

In [20]:
# Calculate the difference
difference = holdout_costs_with_imputation.sub(holdout_costs)

# Calculate percentage difference
percentage_difference = np.abs((difference / cost_df['Cost']) * 100)

# Drop NaN values to keep only Systems with meaningful percentage differences
percentage_difference = percentage_difference.dropna()
percentage_difference

System
AmoebaNet-A (F=448)    2555.912899
GPT-4                     0.000000
GShard (dense)            0.000000
OPT-175B                 50.276243
RoBERTa Large            64.462810
Name: Cost, dtype: float64

In [21]:
percentage_difference.mean()

534.1303904795897

## Scaling up

In [32]:
def abs_ooms_difference(series1, series2):
    return np.abs(np.log10(series1).sub(np.log10(series2)))

def abs_percentage_difference(series1, series2):
    difference = series1.sub(series2)
    return np.abs((difference / series2) * 100)

In [54]:
target_col = 'Training hardware'
reference_col = 'Cost'
imputer_fn = knn_impute_categorical_column
difference_metric = abs_ooms_difference
num_drop = 5
num_trials = 1000
print(f'Running {num_trials} trials of {num_drop} dropped values')
print()
differences = []
for i in range(num_trials):
    frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation()
    frontier_pcd_df = frontier_pcd_df.set_index('System')
    frontier_pcd_df['System'] = frontier_pcd_df.index

    # Drop random values where all of the required information was known
    dropped_df, holdout_df = drop_random_values(frontier_pcd_df, target_col, cost_df, reference_col, num_drop=num_drop)

    # Estimate costs with imputation
    cost_df_with_imputation = estimate_costs(dropped_df, hardware_df, price_df, impute_pcd_data=True)

    # Measure the difference in cost for rows where data was dropped and imputed
    holdout_costs_with_imputation = cost_df_with_imputation.loc[holdout_df.index, reference_col]
    holdout_costs = cost_df.loc[holdout_df.index, reference_col]

    # Calculate the difference
    difference = difference_metric(holdout_costs_with_imputation, holdout_costs)
    differences.extend(difference.dropna().to_list())
print()
print(f'Mean {difference_metric.__name__}: {np.mean(differences)}')
print(f'Std {difference_metric.__name__}: {np.std(differences)}')
print(f'2.5%, 50%, 97.5%: {np.percentile(differences, [2.5, 50, 97.5])}')

Running 2 trials of 5 dropped values

==== System: AlphaGo Fan ====
Trying Google TPU v3 at 2015-07-17 09:00:00
Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (1-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (on-demand)
Found price: 2.0

==== System: AlphaGo Lee ====
Trying Google TPU v3 at 2015-11-12 09:00:00
Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (1-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (on-demand)
Found price: 2.0

==== System: GNMT ====
Trying NVIDIA Tesla K80 at 2016-01-30 00:00:00
Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (1-year CUD)
Could not find price
Trying Google Cloud, Price per chip-hour (on-demand)
Found price: 1.4

==== System: NASv3 (CIFAR-10) ====
Trying Google TPU v3 at 2016-08-21 09:00:00
Tr